In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'daibetes:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3244240%2F5644375%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T180349Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D400e02078b5574695a9db60e30561593e4e838c880fe00702828a6e402a5536863d998c7cad4ba8b8b8def272492c0170de408e86fdf70d02feff23be47974623ccd45e879eff8b57aea90eae966e10e51ba9c319ef6f6b44ceea29101dfbd05ab081353d12bcd647b1fd42467d73bb5ef29d011a6171af4e68d05813274c61e126987781cfeae248b2ea8fe3b27ff844841d46dda64d14a1c54ec493442c78e42fe6a3539a78a122fb4ead62e17b2bc21860bfa1ccb77a6d6a1aaefb3de0349adf2ae0dcda44e9915a90b945fe1917d2e9e4360438b318eedbc0700e3a52bc52e8c6958cb1bd0549a69717f3297791c508ad46af096c7eedb11f56e16e23549'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/daibetes/diabetes (1).csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import keras_tuner as kt
from keras.layers import Dropout
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping

In [ ]:
df=pd.read_csv("/kaggle/input/daibetes/diabetes (1).csv")

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled= sc.transform(X_test)

In [ ]:
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1:]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=12)

In [ ]:
df.duplicated().sum()

0

In [ ]:
def build_model(hp):
    model=Sequential()

    counter=0

    for i in range(hp.Int("layers_name",min_value=1, max_value=10,step=1)):

        if counter == 0:

            model.add(Dense(units=hp.Int("unit" + str(i),min_value=8,max_value=132,step=8),
                            activation=hp.Choice("activation" + str(i),values=["relu","tanh","sigmoid"])
                            ,input_dim=8))
            model.add(Dropout(hp.Choice("Dropout",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(units=hp.Int("unit" + str(i),min_value=8,max_value=132,step=8),
                            activation=hp.Choice("activation"+  str(i),values=["relu","tanh","sigmoid"])))
            model.add(Dropout(hp.Choice("Dropout",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1

    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer=hp.Choice("optimizer",values=["adam","adadelta","sgd","rmsprop"]),loss="binary_crossentropy",metrics=["accuracy"])

    return model

In [ ]:
tunner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=3)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tunner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=10)

In [ ]:
tunner.get_best_hyperparameters()[0].values

{'layers_name': 8,
 'unit0': 40,
 'activation0': 'relu',
 'optimizer': 'rmsprop',
 'unit1': 120,
 'activation1': 'sigmoid',
 'unit2': 120,
 'activation2': 'tanh',
 'unit3': 56,
 'activation3': 'sigmoid',
 'unit4': 80,
 'activation4': 'sigmoid',
 'unit5': 120,
 'activation5': 'relu',
 'unit6': 64,
 'activation6': 'tanh',
 'unit7': 8,
 'activation7': 'relu'}

In [ ]:
model=tunner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=500,callbacks=calback)

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8071 - loss: 0.3841 - val_accuracy: 0.7338 - val_loss: 0.6342
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8124 - loss: 0.4151 - val_accuracy: 0.7403 - val_loss: 0.6119
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7865 - loss: 0.4383 - val_accuracy: 0.7597 - val_loss: 0.6408


In [ ]:
calback=EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=0,
                              verbose=0, mode='auto')